In [2]:
# Check if CUDA recognized

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"GPU count: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("❌ CUDA not detected by PyTorch")

PyTorch version: 2.5.1+cu121
CUDA available: True
CUDA version: 12.1
GPU count: 1
GPU name: NVIDIA GeForce RTX 2060
GPU memory: 6.0 GB


In [3]:
# Broad model family search

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import timm
import numpy as np
import cv2
import os
import gc
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Mixed precision imports (from your reference code)
try:
    from torch.cuda.amp import autocast, GradScaler
    MIXED_PRECISION_AVAILABLE = True
except ImportError:
    MIXED_PRECISION_AVAILABLE = False
    class autocast:
        def __enter__(self):
            return self
        def __exit__(self, *args):
            pass

# GPU Configuration (matching your reference code)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    torch.backends.cudnn.benchmark = True
    
    if MIXED_PRECISION_AVAILABLE:
        scaler = GradScaler()
        use_amp = True
        print("Mixed Precision: Enabled")
    else:
        use_amp = False
        print("Mixed Precision: Disabled")
else:
    use_amp = False
    print("WARNING: GPU not available")

# Data paths
color_path = r"G:\Dropbox\AI Projects\buck\images\squared\color"
grayscale_path = r"G:\Dropbox\AI Projects\buck\images\squared\grayscale"

def parse_filename(filename):
    parts = filename.split('_')
    if len(parts) >= 4:
        age_str = parts[3]
        try:
            age = float(age_str.replace('p', '.'))
            # Cap ages over 5.5 to 5.5
            if age > 5.5:
                age = 5.5
            return age
        except ValueError:
            # Skip files with non-numeric age (e.g., "xpx")
            return None
    return None

def age_to_class(age):
    age_mapping = {1.5: 0, 2.5: 1, 3.5: 2, 4.5: 3, 5.5: 4}
    return age_mapping.get(age, None)

def load_images(color_path, grayscale_path, img_size=(224, 224)):
    images = []
    ages = []
    
    # Process color images (convert to grayscale)
    if os.path.exists(color_path):
        for filename in os.listdir(color_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                age = parse_filename(filename)
                if age is not None:
                    class_idx = age_to_class(age)
                    if class_idx is not None:
                        img_path = os.path.join(color_path, filename)
                        img = cv2.imread(img_path)
                        if img is not None:
                            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                            img_resized = cv2.resize(img_gray, img_size)
                            assert img_resized.shape == img_size, f"Image {filename} not resized correctly: {img_resized.shape}"
                            # Convert to 3-channel for pretrained models
                            img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
                            images.append(img_rgb)
                            ages.append(class_idx)
    
    # Process grayscale images
    if os.path.exists(grayscale_path):
        for filename in os.listdir(grayscale_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                age = parse_filename(filename)
                if age is not None:
                    class_idx = age_to_class(age)
                    if class_idx is not None:
                        img_path = os.path.join(grayscale_path, filename)
                        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                        if img is not None:
                            img_resized = cv2.resize(img, img_size)
                            assert img_resized.shape == img_size, f"Image {filename} not resized correctly: {img_resized.shape}"
                            # Convert to 3-channel for pretrained models
                            img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
                            images.append(img_rgb)
                            ages.append(class_idx)
    
    images = np.array(images)
    ages = np.array(ages)
    
    # Verify final dimensions
    assert images.shape[1:3] == img_size, f"Final image dimensions incorrect: {images.shape}"
    print(f"Images loaded with shape: {images.shape}")
    print(f"Classes: {np.unique(ages)} (0=1.5yr, 1=2.5yr, 2=3.5yr, 3=4.5yr, 4=5.5yr)")
    print(f"Class distribution: {Counter(ages)}")
    
    return images, ages

class DeerDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.LongTensor(y)
        self.mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        self.std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        image = self.X[idx].clone()
        label = self.y[idx].clone()
        
        # Normalize to [0,1]
        if image.max() > 1.0:
            image = image / 255.0
        
        # Convert to CHW format
        if len(image.shape) == 3 and image.shape[-1] == 3:
            image = image.permute(2, 0, 1)
        
        # Normalize with ImageNet stats
        image = (image - self.mean) / self.std
        
        return image, label

def create_model(model_name, num_classes=5):
    """Create model using timm (matching your reference code)"""
    if model_name == 'ResNet50':
        model = timm.create_model('resnet50', pretrained=True, num_classes=num_classes)
    elif model_name == 'EfficientNetB0':
        model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=num_classes)
    elif model_name == 'VGG16':
        model = timm.create_model('vgg16', pretrained=True, num_classes=num_classes)
    elif model_name == 'MobileNetV2':
        model = timm.create_model('mobilenetv2_100', pretrained=True, num_classes=num_classes)
    elif model_name == 'InceptionV3':
        model = timm.create_model('inception_v3', pretrained=True, num_classes=num_classes)
    elif model_name == 'DenseNet121':
        model = timm.create_model('densenet121', pretrained=True, num_classes=num_classes)
    elif model_name == 'ResNet101':
        model = timm.create_model('resnet101', pretrained=True, num_classes=num_classes)
    elif model_name == 'ResNet152':
        model = timm.create_model('resnet152', pretrained=True, num_classes=num_classes)
    elif model_name == 'EfficientNetB1':
        model = timm.create_model('efficientnet_b1', pretrained=True, num_classes=num_classes)
    elif model_name == 'EfficientNetB2':
        model = timm.create_model('efficientnet_b2', pretrained=True, num_classes=num_classes)
    else:
        raise ValueError(f"Unknown model: {model_name}")
    
    return model.to(device)

def train_model(model, train_loader, test_loader, model_name, epochs=50):
    """Train model with your proven GPU configuration"""
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5, min_lr=1e-6)
    
    best_acc = 0.0
    patience = 10
    patience_counter = 0
    best_state = None
    
    print(f"Training {model_name} on {device}")
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_correct = 0
        train_total = 0
        train_loss_total = 0.0
        
        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            
            if use_amp:
                with autocast():
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            
            _, predicted = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            train_loss_total += loss.item()
            
            # Memory management (from your reference code)
            if batch_idx % 10 == 0 and torch.cuda.is_available():
                torch.cuda.empty_cache()
        
        train_acc = 100 * train_correct / train_total
        
        # Validation
        model.eval()
        test_correct = 0
        test_total = 0
        
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                
                if use_amp:
                    with autocast():
                        outputs = model(images)
                else:
                    outputs = model(images)
                
                _, predicted = torch.max(outputs, 1)
                test_total += labels.size(0)
                test_correct += (predicted == labels).sum().item()
        
        test_acc = 100 * test_correct / test_total
        scheduler.step(test_acc)
        
        # Early stopping
        if test_acc > best_acc:
            best_acc = test_acc
            patience_counter = 0
            best_state = model.state_dict().copy()
        else:
            patience_counter += 1
        
        if epoch % 10 == 0 or patience_counter >= patience:
            print(f"  Epoch {epoch:2d}: Train {train_acc:.1f}%, Test {test_acc:.1f}%")
        
        if patience_counter >= patience:
            print(f"  Early stopping at epoch {epoch}")
            break
        
        # Memory cleanup
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    # Restore best model
    if best_state is not None:
        model.load_state_dict(best_state)
    
    return model, best_acc

# Create timestamped output folder
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f"deer_age_models_{timestamp}"
os.makedirs(output_dir, exist_ok=True)
print(f"Models will be saved to: {output_dir}")

# Load data
print("Loading images...")
X, y = load_images(color_path, grayscale_path)
print(f"Loaded {len(X)} images, age range: {y.min():.1f}-{y.max():.1f}")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train: {len(X_train)}, Test: {len(X_test)}")

# Create datasets
train_dataset = DeerDataset(X_train, y_train)
test_dataset = DeerDataset(X_test, y_test)

# Create dataloaders (using your batch size from reference code)
batch_size = 16  # From your reference code for RTX 2060
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# Model families to test
model_families = [
    'ResNet50', 'EfficientNetB0', 'VGG16', 
    'MobileNetV2', 'InceptionV3', 'DenseNet121'
]

results = []
best_accuracy = 0
best_family = None

print(f"\nTesting {len(model_families)} model families...")

for model_name in model_families:
    print(f"\nTesting {model_name}...")
    
    try:
        model = create_model(model_name)
        trained_model, accuracy = train_model(
            model, train_loader, test_loader, model_name
        )
        
        # Save model with accuracy in filename
        acc_str = f"{accuracy:.3f}".replace('.', 'p')
        model_filename = f"{model_name}_{acc_str}.pth"
        model_path = os.path.join(output_dir, model_filename)
        
        torch.save({
            'model_state_dict': trained_model.state_dict(),
            'model_name': model_name,
            'accuracy': accuracy,
            'num_classes': 5
        }, model_path)
        
        results.append({
            'model': model_name,
            'accuracy': accuracy,
            'filename': model_filename,
            'full_path': model_path
        })
        
        print(f"{model_name}: Accuracy={accuracy:.3f}, Saved: {model_filename}")
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_family = model_name
        
        # Cleanup (from your reference code)
        del model, trained_model
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()
        
    except Exception as e:
        print(f"Error with {model_name}: {e}")
        continue

# Find best performing family
print(f"\nBest family: {best_family} (accuracy: {best_accuracy:.3f})")

# Test variations within best family
if best_family:
    print(f"\nTesting variations of {best_family}...")
    
    if best_family == 'ResNet50':
        variations = ['ResNet101', 'ResNet152']
    elif best_family == 'EfficientNetB0':
        variations = ['EfficientNetB1', 'EfficientNetB2']
    else:
        variations = []
    
    for var_name in variations:
        print(f"\nTesting {var_name}...")
        
        try:
            model = create_model(var_name)
            trained_model, accuracy = train_model(
                model, train_loader, test_loader, var_name
            )
            
            # Save model
            acc_str = f"{accuracy:.3f}".replace('.', 'p')
            model_filename = f"{var_name}_{acc_str}.pth"
            model_path = os.path.join(output_dir, model_filename)
            
            torch.save({
                'model_state_dict': trained_model.state_dict(),
                'model_name': var_name,
                'accuracy': accuracy,
                'num_classes': 5
            }, model_path)
            
            results.append({
                'model': var_name,
                'accuracy': accuracy,
                'filename': model_filename,
                'full_path': model_path
            })
            
            print(f"{var_name}: Accuracy={accuracy:.3f}, Saved: {model_filename}")
            
            # Cleanup
            del model, trained_model
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            gc.collect()
            
        except Exception as e:
            print(f"Error with {var_name}: {e}")
            continue

# Final results
print(f"\n{'='*50}")
print("FINAL RESULTS")
print(f"{'='*50}")

results.sort(key=lambda x: x['accuracy'], reverse=True)

for i, result in enumerate(results, 1):
    print(f"{i:2d}. {result['model']:15s} - Accuracy: {result['accuracy']:.3f}")

if results:
    best_result = results[0]
    print(f"\nBest model: {best_result['model']} with {best_result['accuracy']:.3f} accuracy")
    print(f"Saved as: {best_result['filename']}")

print(f"\nTotal models tested: {len(results)}")
print(f"All models saved in folder: {output_dir}")
print("All models saved with accuracy in filename.")

Using device: cuda
GPU: NVIDIA GeForce RTX 2060
GPU Memory: 6.0 GB
Mixed Precision: Enabled
Models will be saved to: deer_age_models_20250731_072915
Loading images...
Images loaded with shape: (466, 224, 224, 3)
Classes: [0 1 2 3 4] (0=1.5yr, 1=2.5yr, 2=3.5yr, 3=4.5yr, 4=5.5yr)
Class distribution: Counter({np.int64(4): 118, np.int64(2): 111, np.int64(3): 89, np.int64(1): 82, np.int64(0): 66})
Loaded 466 images, age range: 0.0-4.0
Train: 372, Test: 94

Testing 6 model families...

Testing ResNet50...
Training ResNet50 on cuda
  Epoch  0: Train 22.6%, Test 25.5%
  Epoch 10: Train 100.0%, Test 47.9%
  Epoch 19: Train 100.0%, Test 50.0%
  Early stopping at epoch 19
ResNet50: Accuracy=51.064, Saved: ResNet50_51p064.pth

Testing EfficientNetB0...
Training EfficientNetB0 on cuda
  Epoch  0: Train 29.6%, Test 35.1%
  Epoch 10: Train 100.0%, Test 50.0%
  Epoch 18: Train 100.0%, Test 55.3%
  Early stopping at epoch 18
EfficientNetB0: Accuracy=55.319, Saved: EfficientNetB0_55p319.pth

Testing VGG

In [5]:
# Deeper family search.

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import timm
import numpy as np
import cv2
import os
import gc
import random
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

try:
    from torch.cuda.amp import autocast, GradScaler
    MIXED_PRECISION_AVAILABLE = True
except ImportError:
    MIXED_PRECISION_AVAILABLE = False
    class autocast:
        def __enter__(self):
            return self
        def __exit__(self, *args):
            pass

# GPU Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    torch.backends.cudnn.benchmark = True
    
    if MIXED_PRECISION_AVAILABLE:
        scaler = GradScaler()
        use_amp = True
        print("Mixed Precision: Enabled")
    else:
        use_amp = False
else:
    use_amp = False

# Data paths
color_path = r"G:\Dropbox\AI Projects\buck\images\squared\color"
grayscale_path = r"G:\Dropbox\AI Projects\buck\images\squared\grayscale"

def parse_filename(filename):
    parts = filename.split('_')
    if len(parts) >= 4:
        age_str = parts[3]
        try:
            age = float(age_str.replace('p', '.'))
            if age > 5.5:
                age = 5.5
            return age
        except ValueError:
            return None
    return None

def age_to_class(age):
    age_mapping = {1.5: 0, 2.5: 1, 3.5: 2, 4.5: 3, 5.5: 4}
    return age_mapping.get(age, None)

def augment_image(image):
    """Enhanced augmentation to reduce overfitting"""
    if random.random() < 0.5:
        image = cv2.flip(image, 1)
    
    if random.random() < 0.7:
        angle = random.uniform(-15, 15)
        h, w = image.shape[:2]
        M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1.0)
        image = cv2.warpAffine(image, M, (w, h))
    
    if random.random() < 0.8:
        alpha = random.uniform(0.7, 1.3)
        beta = random.randint(-25, 25)
        image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    
    if random.random() < 0.3:
        noise = np.random.normal(0, 8, image.shape).astype(np.int16)
        image_int16 = image.astype(np.int16)
        noisy_image = np.clip(image_int16 + noise, 0, 255)
        image = noisy_image.astype(np.uint8)
    
    return image

def load_images(color_path, grayscale_path, img_size=(224, 224)):
    images = []
    ages = []
    
    if os.path.exists(color_path):
        for filename in os.listdir(color_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                age = parse_filename(filename)
                if age is not None:
                    class_idx = age_to_class(age)
                    if class_idx is not None:
                        img_path = os.path.join(color_path, filename)
                        img = cv2.imread(img_path)
                        if img is not None:
                            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                            img_resized = cv2.resize(img_gray, img_size)
                            img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
                            images.append(img_rgb)
                            ages.append(class_idx)
    
    if os.path.exists(grayscale_path):
        for filename in os.listdir(grayscale_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                age = parse_filename(filename)
                if age is not None:
                    class_idx = age_to_class(age)
                    if class_idx is not None:
                        img_path = os.path.join(grayscale_path, filename)
                        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                        if img is not None:
                            img_resized = cv2.resize(img, img_size)
                            img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
                            images.append(img_rgb)
                            ages.append(class_idx)
    
    images = np.array(images)
    ages = np.array(ages)
    
    print(f"Images loaded with shape: {images.shape}")
    print(f"Class distribution: {Counter(ages)}")
    
    return images, ages

class AugmentedDeerDataset(Dataset):
    def __init__(self, X, y, augment=False):
        self.X = X
        self.y = torch.LongTensor(y)
        self.augment = augment
        self.mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        self.std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        image = self.X[idx].copy()
        label = self.y[idx].clone()
        
        # Apply augmentation during training
        if self.augment:
            image = augment_image(image)
        
        # Convert to tensor and normalize
        image = torch.FloatTensor(image)
        if image.max() > 1.0:
            image = image / 255.0
        
        if len(image.shape) == 3 and image.shape[-1] == 3:
            image = image.permute(2, 0, 1)
        
        image = (image - self.mean) / self.std
        
        return image, label

def create_model_with_regularization(model_name, num_classes=5, dropout_rate=0.5):
    """Create model with better regularization"""
    model = timm.create_model(model_name, pretrained=True, num_classes=num_classes, drop_rate=dropout_rate)
    
    # Freeze more layers to reduce overfitting
    if 'resnet' in model_name:
        for name, param in model.named_parameters():
            if not ('layer4' in name or 'fc' in name):
                param.requires_grad = False
    elif 'efficientnet' in model_name:
        for name, param in model.named_parameters():
            if not ('blocks.6' in name or 'blocks.7' in name or 'classifier' in name):
                param.requires_grad = False
    elif 'densenet' in model_name:
        for name, param in model.named_parameters():
            if not ('denseblock4' in name or 'classifier' in name):
                param.requires_grad = False
    elif 'mobilenet' in model_name:
        for name, param in model.named_parameters():
            if not ('features.18' in name or 'features.19' in name or 'classifier' in name):
                param.requires_grad = False
    
    return model.to(device)

def train_model_improved(model, train_loader, test_loader, model_name, epochs=60):
    """Improved training with better regularization"""
    criterion = nn.CrossEntropyLoss(label_smoothing=0.15)
    
    # Lower learning rate and higher weight decay
    optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=0.05)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-6)
    
    best_acc = 0.0
    patience = 15
    patience_counter = 0
    best_state = None
    
    print(f"Training {model_name} with improved regularization")
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_correct = 0
        train_total = 0
        
        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            
            if use_amp:
                with autocast():
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            
            _, predicted = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            
            if batch_idx % 10 == 0 and torch.cuda.is_available():
                torch.cuda.empty_cache()
        
        train_acc = 100 * train_correct / train_total
        scheduler.step()
        
        # Validation
        model.eval()
        test_correct = 0
        test_total = 0
        
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                
                if use_amp:
                    with autocast():
                        outputs = model(images)
                else:
                    outputs = model(images)
                
                _, predicted = torch.max(outputs, 1)
                test_total += labels.size(0)
                test_correct += (predicted == labels).sum().item()
        
        test_acc = 100 * test_correct / test_total
        
        # Early stopping
        if test_acc > best_acc:
            best_acc = test_acc
            patience_counter = 0
            best_state = model.state_dict().copy()
        else:
            patience_counter += 1
        
        if epoch % 10 == 0 or patience_counter >= patience:
            print(f"  Epoch {epoch:2d}: Train {train_acc:.1f}%, Test {test_acc:.1f}%")
        
        if patience_counter >= patience:
            print(f"  Early stopping at epoch {epoch}")
            break
        
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    if best_state is not None:
        model.load_state_dict(best_state)
    
    return model, best_acc

# Create timestamped output folder
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f"deer_age_deep_survey_{timestamp}"
os.makedirs(output_dir, exist_ok=True)
print(f"Models will be saved to: {output_dir}")

# Load data
print("Loading images...")
X, y = load_images(color_path, grayscale_path)
print(f"Loaded {len(X)} images")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train: {len(X_train)}, Test: {len(X_test)}")

# Create datasets with augmentation
train_dataset = AugmentedDeerDataset(X_train, y_train, augment=True)
test_dataset = AugmentedDeerDataset(X_test, y_test, augment=False)

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# Deep exploration of top 3 families
model_configs = [
    # DenseNet family (won previous round)
    ('densenet121', 'DenseNet121'),
    ('densenet169', 'DenseNet169'),
    ('densenet201', 'DenseNet201'),
    
    # EfficientNet family (2nd place)
    ('efficientnet_b0', 'EfficientNetB0'),
    ('efficientnet_b1', 'EfficientNetB1'),
    ('efficientnet_b2', 'EfficientNetB2'),
    ('efficientnet_b3', 'EfficientNetB3'),
    
    # MobileNet family (3rd place)
    ('mobilenetv2_100', 'MobileNetV2'),
    ('mobilenetv3_small_100', 'MobileNetV3Small'),
    ('mobilenetv3_large_100', 'MobileNetV3Large'),
    
    # Additional high-performers to test
    ('resnet50', 'ResNet50_Regularized'),
    ('resnext50_32x4d', 'ResNeXt50'),
]

results = []
print(f"\nDeep survey: Testing {len(model_configs)} models with improved regularization...")

for model_timm_name, display_name in model_configs:
    print(f"\nTesting {display_name}...")
    
    try:
        model = create_model_with_regularization(model_timm_name, dropout_rate=0.5)
        trained_model, accuracy = train_model_improved(
            model, train_loader, test_loader, display_name
        )
        
        # Save model
        acc_str = f"{accuracy:.3f}".replace('.', 'p')
        model_filename = f"{display_name}_{acc_str}.pth"
        model_path = os.path.join(output_dir, model_filename)
        
        torch.save({
            'model_state_dict': trained_model.state_dict(),
            'model_name': display_name,
            'timm_name': model_timm_name,
            'accuracy': accuracy,
            'num_classes': 5
        }, model_path)
        
        results.append({
            'model': display_name,
            'timm_name': model_timm_name,
            'accuracy': accuracy,
            'filename': model_filename,
            'full_path': model_path
        })
        
        print(f"{display_name}: {accuracy:.3f}% - Saved: {model_filename}")
        
        # Cleanup
        del model, trained_model
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()
        
    except Exception as e:
        print(f"Error with {display_name}: {e}")
        continue

# Final results
print(f"\n{'='*60}")
print("DEEP SURVEY RESULTS - TOP 3 FAMILIES + EXTRAS")
print(f"{'='*60}")

results.sort(key=lambda x: x['accuracy'], reverse=True)

print("DENSENET FAMILY:")
for result in results:
    if 'DenseNet' in result['model']:
        print(f"  {result['model']:20s} - {result['accuracy']:.3f}%")

print("\nEFFICIENTNET FAMILY:")
for result in results:
    if 'EfficientNet' in result['model']:
        print(f"  {result['model']:20s} - {result['accuracy']:.3f}%")

print("\nMOBILENET FAMILY:")
for result in results:
    if 'MobileNet' in result['model']:
        print(f"  {result['model']:20s} - {result['accuracy']:.3f}%")

print("\nOTHER MODELS:")
for result in results:
    if not any(family in result['model'] for family in ['DenseNet', 'EfficientNet', 'MobileNet']):
        print(f"  {result['model']:20s} - {result['accuracy']:.3f}%")

print(f"\n{'='*60}")
print("OVERALL RANKING:")
for i, result in enumerate(results, 1):
    print(f"{i:2d}. {result['model']:20s} - {result['accuracy']:.3f}%")

if results:
    best_result = results[0]
    print(f"\nBEST MODEL: {best_result['model']} - {best_result['accuracy']:.3f}%")
    print(f"Saved as: {best_result['filename']}")

print(f"\nTotal models tested: {len(results)}")
print(f"All models saved in: {output_dir}")
print("Note: Improved regularization should reduce train/test accuracy gap")

Using device: cuda
GPU: NVIDIA GeForce RTX 2060
GPU Memory: 6.0 GB
Mixed Precision: Enabled
Models will be saved to: deer_age_deep_survey_20250731_075454
Loading images...
Images loaded with shape: (466, 224, 224, 3)
Class distribution: Counter({np.int64(4): 118, np.int64(2): 111, np.int64(3): 89, np.int64(1): 82, np.int64(0): 66})
Loaded 466 images
Train: 372, Test: 94

Deep survey: Testing 12 models with improved regularization...

Testing DenseNet121...
Training DenseNet121 with improved regularization
  Epoch  0: Train 25.5%, Test 30.9%
  Epoch 10: Train 85.8%, Test 56.4%
  Epoch 20: Train 91.1%, Test 46.8%
  Epoch 25: Train 95.7%, Test 47.9%
  Early stopping at epoch 25
DenseNet121: 56.383% - Saved: DenseNet121_56p383.pth

Testing DenseNet169...
Training DenseNet169 with improved regularization
  Epoch  0: Train 27.7%, Test 34.0%
  Epoch 10: Train 90.3%, Test 52.1%
  Epoch 20: Train 95.7%, Test 60.6%
  Epoch 30: Train 98.9%, Test 60.6%
  Epoch 40: Train 99.7%, Test 63.8%
  Epoch 4

In [7]:
# Second attempt at model families 

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import timm
import numpy as np
import cv2
import os
import gc
import random
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

try:
    from torch.cuda.amp import autocast, GradScaler
    MIXED_PRECISION_AVAILABLE = True
except ImportError:
    MIXED_PRECISION_AVAILABLE = False
    class autocast:
        def __enter__(self):
            return self
        def __exit__(self, *args):
            pass

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    torch.backends.cudnn.benchmark = True
    if MIXED_PRECISION_AVAILABLE:
        scaler = GradScaler()
        use_amp = True
        print("Mixed Precision: Enabled")
    else:
        use_amp = False
else:
    use_amp = False

color_path = r"G:\Dropbox\AI Projects\buck\images\squared\color"
grayscale_path = r"G:\Dropbox\AI Projects\buck\images\squared\grayscale"

def parse_filename(filename):
    parts = filename.split('_')
    if len(parts) >= 4:
        age_str = parts[3]
        try:
            age = float(age_str.replace('p', '.'))
            if age > 5.5:
                age = 5.5
            return age
        except ValueError:
            return None
    return None

def age_to_class(age):
    age_mapping = {1.5: 0, 2.5: 1, 3.5: 2, 4.5: 3, 5.5: 4}
    return age_mapping.get(age, None)

def mixup_data(x, y, alpha=0.4):
    """Mixup augmentation to create synthetic training examples"""
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    
    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(device)
    
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    """Mixup loss function"""
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

def load_images(color_path, grayscale_path, img_size=(224, 224)):
    images = []
    ages = []
    
    if os.path.exists(color_path):
        for filename in os.listdir(color_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                age = parse_filename(filename)
                if age is not None:
                    class_idx = age_to_class(age)
                    if class_idx is not None:
                        img_path = os.path.join(color_path, filename)
                        img = cv2.imread(img_path)
                        if img is not None:
                            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                            img_resized = cv2.resize(img_gray, img_size)
                            img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
                            images.append(img_rgb)
                            ages.append(class_idx)
    
    if os.path.exists(grayscale_path):
        for filename in os.listdir(grayscale_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                age = parse_filename(filename)
                if age is not None:
                    class_idx = age_to_class(age)
                    if class_idx is not None:
                        img_path = os.path.join(grayscale_path, filename)
                        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                        if img is not None:
                            img_resized = cv2.resize(img, img_size)
                            img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
                            images.append(img_rgb)
                            ages.append(class_idx)
    
    images = np.array(images)
    ages = np.array(ages)
    
    print(f"Total images: {len(images)}")
    print(f"Class distribution: {Counter(ages)}")
    
    return images, ages

def conservative_augment(image):
    """Very light augmentation to preserve deer features"""
    if random.random() < 0.5:
        image = cv2.flip(image, 1)
    
    if random.random() < 0.3:
        angle = random.uniform(-8, 8)
        h, w = image.shape[:2]
        M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1.0)
        image = cv2.warpAffine(image, M, (w, h))
    
    if random.random() < 0.4:
        alpha = random.uniform(0.9, 1.1)
        beta = random.randint(-10, 10)
        image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    
    return image

class MultiScaleDataset(Dataset):
    def __init__(self, X, y, augment=False, scale_size=224):
        self.X = X
        self.y = torch.LongTensor(y)
        self.augment = augment
        self.scale_size = scale_size
        self.mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        self.std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        image = self.X[idx].copy()
        label = self.y[idx].clone()
        
        # Multi-scale training
        if self.augment:
            scale_factor = random.choice([0.8, 0.9, 1.0, 1.1, 1.2])
            new_size = int(self.scale_size * scale_factor)
            image = cv2.resize(image, (new_size, new_size))
            image = cv2.resize(image, (self.scale_size, self.scale_size))
            
            image = conservative_augment(image)
        
        image = torch.FloatTensor(image)
        if image.max() > 1.0:
            image = image / 255.0
        
        if len(image.shape) == 3 and image.shape[-1] == 3:
            image = image.permute(2, 0, 1)
        
        image = (image - self.mean) / self.std
        
        return image, label

def create_conservative_model(model_name, num_classes=5):
    """Back to simpler model creation that worked"""
    model = timm.create_model(model_name, pretrained=True, num_classes=num_classes, drop_rate=0.4)
    
    # Conservative freezing (like the 63.8% model)
    if 'densenet' in model_name:
        for name, param in model.named_parameters():
            if not ('denseblock4' in name or 'classifier' in name):
                param.requires_grad = False
    elif 'resnext' in model_name or 'resnet' in model_name:
        for name, param in model.named_parameters():
            if not ('layer4' in name or 'fc' in name):
                param.requires_grad = False
    
    return model.to(device)

def train_with_mixup_and_multiscale(model, train_loader, test_loader, model_name, epochs=120):
    """Training with mixup + multi-scale + very conservative approach"""
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    
    # Conservative optimizer (back to what worked)
    optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=0.05)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-6)
    
    best_acc = 0.0
    patience = 30
    patience_counter = 0
    best_state = None
    
    print(f"Training {model_name} with Mixup + Multi-scale")
    
    for epoch in range(epochs):
        # Training with mixup
        model.train()
        train_correct = 0
        train_total = 0
        
        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            
            # Apply mixup
            if random.random() < 0.5:  # 50% chance of mixup
                mixed_images, y_a, y_b, lam = mixup_data(images, labels, alpha=0.4)
                optimizer.zero_grad()
                
                if use_amp:
                    with autocast():
                        outputs = model(mixed_images)
                        loss = mixup_criterion(criterion, outputs, y_a, y_b, lam)
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    outputs = model(mixed_images)
                    loss = mixup_criterion(criterion, outputs, y_a, y_b, lam)
                    loss.backward()
                    optimizer.step()
                
                # For accuracy calculation, use original labels
                _, predicted = torch.max(outputs, 1)
                train_total += labels.size(0)
                train_correct += (predicted == y_a).sum().item()
            else:
                # Normal training
                optimizer.zero_grad()
                
                if use_amp:
                    with autocast():
                        outputs = model(images)
                        loss = criterion(outputs, labels)
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                
                _, predicted = torch.max(outputs, 1)
                train_total += labels.size(0)
                train_correct += (predicted == labels).sum().item()
            
            if batch_idx % 10 == 0 and torch.cuda.is_available():
                torch.cuda.empty_cache()
        
        train_acc = 100 * train_correct / train_total
        scheduler.step()
        
        # Simple TTA evaluation (not too heavy)
        test_acc = evaluate_with_simple_tta(model, test_loader)
        
        if test_acc > best_acc:
            best_acc = test_acc
            patience_counter = 0
            best_state = model.state_dict().copy()
        else:
            patience_counter += 1
        
        if epoch % 20 == 0 or patience_counter >= patience:
            print(f"  Epoch {epoch:3d}: Train {train_acc:.1f}%, Test+TTA {test_acc:.1f}%")
        
        if patience_counter >= patience:
            print(f"  Early stopping at epoch {epoch}")
            break
        
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    if best_state is not None:
        model.load_state_dict(best_state)
    
    return model, best_acc

def evaluate_with_simple_tta(model, test_loader):
    """Simple TTA - just 3 versions"""
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            
            # Original prediction
            if use_amp:
                with autocast():
                    outputs1 = model(images)
            else:
                outputs1 = model(images)
            
            # Horizontal flip
            flipped = torch.flip(images, [3])
            if use_amp:
                with autocast():
                    outputs2 = model(flipped)
            else:
                outputs2 = model(flipped)
            
            # Slight zoom
            zoomed = F.interpolate(images, scale_factor=0.95, mode='bilinear', align_corners=False)
            zoomed = F.interpolate(zoomed, size=(224, 224), mode='bilinear', align_corners=False)
            if use_amp:
                with autocast():
                    outputs3 = model(zoomed)
            else:
                outputs3 = model(zoomed)
            
            # Average predictions
            avg_outputs = (F.softmax(outputs1, dim=1) + F.softmax(outputs2, dim=1) + F.softmax(outputs3, dim=1)) / 3
            _, predicted = torch.max(avg_outputs, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    return 100 * correct / total

def ensemble_predict(models, test_loader):
    """Simple ensemble of multiple models"""
    all_models_eval = [model.eval() for model in models]
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            
            ensemble_output = torch.zeros(images.size(0), 5).to(device)
            
            for model in models:
                # Simple TTA for each model
                if use_amp:
                    with autocast():
                        outputs1 = model(images)
                        outputs2 = model(torch.flip(images, [3]))
                else:
                    outputs1 = model(images)
                    outputs2 = model(torch.flip(images, [3]))
                
                avg_model_output = (F.softmax(outputs1, dim=1) + F.softmax(outputs2, dim=1)) / 2
                ensemble_output += avg_model_output
            
            # Final ensemble prediction
            _, predicted = torch.max(ensemble_output, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    return 100 * correct / total

# Main execution
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f"deer_age_ensemble_{timestamp}"
os.makedirs(output_dir, exist_ok=True)
print(f"Ensemble models saved to: {output_dir}")

print("Loading images...")
X, y = load_images(color_path, grayscale_path)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train: {len(X_train)}, Test: {len(X_test)}")

# Create datasets
train_dataset = MultiScaleDataset(X_train, y_train, augment=True)
test_dataset = MultiScaleDataset(X_test, y_test, augment=False)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=0)

print("\n" + "="*60)
print("SMALL DATA STRATEGY: MIXUP + MULTI-SCALE + ENSEMBLE")
print("="*60)
print("Approach: Conservative training + Mixup synthetic data")

# Train multiple models for ensemble
model_configs = [
    ('densenet169', 'DenseNet169'),
    ('resnext50_32x4d', 'ResNeXt50'),
    ('densenet201', 'DenseNet201'),
]

trained_models = []
individual_scores = []

for model_timm_name, display_name in model_configs:
    print(f"\n{'='*40}")
    print(f"Training {display_name}")
    print(f"{'='*40}")
    
    try:
        model = create_conservative_model(model_timm_name)
        trained_model, accuracy = train_with_mixup_and_multiscale(
            model, train_loader, test_loader, display_name
        )
        
        # Save individual model
        acc_str = f"{accuracy:.1f}".replace('.', 'p')
        model_filename = f"{display_name}_{acc_str}pct.pth"
        model_path = os.path.join(output_dir, model_filename)
        
        torch.save({
            'model_state_dict': trained_model.state_dict(),
            'model_name': display_name,
            'timm_name': model_timm_name,
            'accuracy': accuracy,
            'num_classes': 5
        }, model_path)
        
        trained_models.append(trained_model)
        individual_scores.append(accuracy)
        
        print(f"{display_name}: {accuracy:.1f}% - Saved")
        
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
    except Exception as e:
        print(f"Error with {display_name}: {e}")
        continue

# Ensemble evaluation
if len(trained_models) > 1:
    print(f"\n{'='*40}")
    print("ENSEMBLE EVALUATION")
    print(f"{'='*40}")
    
    ensemble_accuracy = ensemble_predict(trained_models, test_loader)
    
    print("INDIVIDUAL MODEL RESULTS:")
    for i, (score, config) in enumerate(zip(individual_scores, model_configs)):
        print(f"  {config[1]}: {score:.1f}%")
    
    print(f"\nENSEMBLE RESULT: {ensemble_accuracy:.1f}%")
    
    if ensemble_accuracy >= 75.0:
        print("SUCCESS: 75% target achieved!")
    else:
        gap = 75.0 - ensemble_accuracy
        print(f"Gap to 75%: {gap:.1f}%")
        
        if ensemble_accuracy > max(individual_scores):
            improvement = ensemble_accuracy - max(individual_scores)
            print(f"Ensemble improvement: +{improvement:.1f}%")

print(f"\nAll models saved in: {output_dir}")
print("="*60)

Using device: cuda
GPU: NVIDIA GeForce RTX 2060
Mixed Precision: Enabled
Ensemble models saved to: deer_age_ensemble_20250731_220115
Loading images...
Total images: 466
Class distribution: Counter({np.int64(4): 118, np.int64(2): 111, np.int64(3): 89, np.int64(1): 82, np.int64(0): 66})
Train: 372, Test: 94

SMALL DATA STRATEGY: MIXUP + MULTI-SCALE + ENSEMBLE
Approach: Conservative training + Mixup synthetic data

Training DenseNet169
Training DenseNet169 with Mixup + Multi-scale
  Epoch   0: Train 29.8%, Test+TTA 37.2%
  Epoch  20: Train 64.2%, Test+TTA 54.3%
  Epoch  40: Train 80.1%, Test+TTA 56.4%
  Epoch  60: Train 78.5%, Test+TTA 62.8%
  Epoch  64: Train 81.5%, Test+TTA 57.4%
  Early stopping at epoch 64
DenseNet169: 64.9% - Saved

Training ResNeXt50
Training ResNeXt50 with Mixup + Multi-scale
  Epoch   0: Train 23.1%, Test+TTA 24.5%
  Epoch  20: Train 84.9%, Test+TTA 53.2%
  Epoch  40: Train 84.7%, Test+TTA 52.1%
  Epoch  58: Train 84.4%, Test+TTA 57.4%
  Early stopping at epoch 58

In [9]:
# Third attempt at model families

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import timm
import numpy as np
import cv2
import os
import gc
import random
import math
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

try:
    from torch.cuda.amp import autocast, GradScaler
    MIXED_PRECISION_AVAILABLE = True
except ImportError:
    MIXED_PRECISION_AVAILABLE = False
    class autocast:
        def __enter__(self):
            return self
        def __exit__(self, *args):
            pass

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    torch.backends.cudnn.benchmark = True
    if MIXED_PRECISION_AVAILABLE:
        scaler = GradScaler()
        use_amp = True
        print("Mixed Precision: Enabled")
    else:
        use_amp = False
else:
    use_amp = False

color_path = r"G:\Dropbox\AI Projects\buck\images\squared\color"
grayscale_path = r"G:\Dropbox\AI Projects\buck\images\squared\grayscale"

def parse_filename(filename):
    parts = filename.split('_')
    if len(parts) >= 4:
        age_str = parts[3]
        try:
            age = float(age_str.replace('p', '.'))
            if age > 5.5:
                age = 5.5
            return age
        except ValueError:
            return None
    return None

def age_to_class(age):
    age_mapping = {1.5: 0, 2.5: 1, 3.5: 2, 4.5: 3, 5.5: 4}
    return age_mapping.get(age, None)

def age_to_ordinal(age):
    """Convert age to ordinal targets for ordinal regression"""
    class_idx = age_to_class(age)
    if class_idx is None:
        return None
    # Create ordinal targets: [1,1,1,0,0] for class 2, [1,1,1,1,0] for class 3, etc.
    ordinal = [1 if i <= class_idx else 0 for i in range(5)]
    return ordinal

def cutmix_data(x, y, alpha=1.0):
    """CutMix augmentation"""
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(device)
    
    W, H = x.size(2), x.size(3)
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)
    
    cx = np.random.randint(W)
    cy = np.random.randint(H)
    
    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    
    x[:, :, bbx1:bbx2, bby1:bby2] = x[index, :, bbx1:bbx2, bby1:bby2]
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (W * H))
    
    return x, y, y[index], lam

def load_images(color_path, grayscale_path, img_size=(224, 224)):
    images = []
    ages = []
    
    if os.path.exists(color_path):
        for filename in os.listdir(color_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                age = parse_filename(filename)
                if age is not None:
                    class_idx = age_to_class(age)
                    if class_idx is not None:
                        img_path = os.path.join(color_path, filename)
                        img = cv2.imread(img_path)
                        if img is not None:
                            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                            img_resized = cv2.resize(img_gray, img_size)
                            img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
                            images.append(img_rgb)
                            ages.append(class_idx)
    
    if os.path.exists(grayscale_path):
        for filename in os.listdir(grayscale_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                age = parse_filename(filename)
                if age is not None:
                    class_idx = age_to_class(age)
                    if class_idx is not None:
                        img_path = os.path.join(grayscale_path, filename)
                        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                        if img is not None:
                            img_resized = cv2.resize(img, img_size)
                            img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
                            images.append(img_rgb)
                            ages.append(class_idx)
    
    images = np.array(images)
    ages = np.array(ages)
    
    print(f"Total images: {len(images)}")
    print(f"Class distribution: {Counter(ages)}")
    
    return images, ages

def advanced_augment(image):
    """Comprehensive augmentation suite"""
    # Random horizontal flip
    if random.random() < 0.6:
        image = cv2.flip(image, 1)
    
    # Random rotation with scaling
    if random.random() < 0.8:
        angle = random.uniform(-20, 20)
        scale = random.uniform(0.9, 1.1)
        h, w = image.shape[:2]
        M = cv2.getRotationMatrix2D((w//2, h//2), angle, scale)
        image = cv2.warpAffine(image, M, (w, h))
    
    # Color jittering
    if random.random() < 0.9:
        alpha = random.uniform(0.8, 1.2)
        beta = random.randint(-20, 20)
        image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    
    # Gaussian noise
    if random.random() < 0.4:
        noise = np.random.normal(0, random.uniform(3, 8), image.shape).astype(np.int16)
        image_int16 = image.astype(np.int16)
        noisy_image = np.clip(image_int16 + noise, 0, 255)
        image = noisy_image.astype(np.uint8)
    
    # Random erasing
    if random.random() < 0.3:
        h, w, c = image.shape
        area = h * w
        target_area = random.uniform(0.02, 0.1) * area
        aspect_ratio = random.uniform(0.3, 3.3)
        
        h_erase = int(round(math.sqrt(target_area * aspect_ratio)))
        w_erase = int(round(math.sqrt(target_area / aspect_ratio)))
        
        if h_erase < h and w_erase < w:
            x1 = random.randint(0, h - h_erase)
            y1 = random.randint(0, w - w_erase)
            image[x1:x1+h_erase, y1:y1+w_erase, :] = random.randint(0, 255)
    
    return image

class AdvancedDataset(Dataset):
    def __init__(self, X, y, augment=False, progressive_size=None, ordinal=False):
        self.X = X
        self.y = torch.LongTensor(y)
        self.augment = augment
        self.progressive_size = progressive_size or 224
        self.ordinal = ordinal
        self.mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        self.std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        image = self.X[idx].copy()
        label = self.y[idx].clone()
        
        # Progressive resizing
        if self.progressive_size != 224:
            image = cv2.resize(image, (self.progressive_size, self.progressive_size))
            image = cv2.resize(image, (224, 224))
        
        if self.augment:
            image = advanced_augment(image)
        
        image = torch.FloatTensor(image)
        if image.max() > 1.0:
            image = image / 255.0
        
        if len(image.shape) == 3 and image.shape[-1] == 3:
            image = image.permute(2, 0, 1)
        
        image = (image - self.mean) / self.std
        
        if self.ordinal:
            # Convert to ordinal targets
            ordinal_targets = torch.FloatTensor([1 if i <= label.item() else 0 for i in range(4)])
            return image, ordinal_targets
        
        return image, label

class OrdinalLoss(nn.Module):
    """Ordinal regression loss"""
    def __init__(self):
        super(OrdinalLoss, self).__init__()
    
    def forward(self, predictions, targets):
        return F.binary_cross_entropy_with_logits(predictions, targets)

class SelfSupervisedPretrainer:
    """Self-supervised pretraining on the deer images"""
    def __init__(self, model):
        self.model = model
        # Replace final layer for rotation prediction (4 classes: 0, 90, 180, 270)
        if hasattr(model, 'classifier'):
            model.classifier = nn.Linear(model.classifier.in_features, 4)
        elif hasattr(model, 'fc'):
            model.fc = nn.Linear(model.fc.in_features, 4)
    
    def create_rotation_dataset(self, images):
        """Create rotation prediction dataset"""
        rotation_images = []
        rotation_labels = []
        
        for img in images:
            for rotation in range(4):
                rotated = np.rot90(img, k=rotation, axes=(0, 1))
                rotation_images.append(rotated)
                rotation_labels.append(rotation)
        
        return np.array(rotation_images), np.array(rotation_labels)
    
    def pretrain(self, images, epochs=50):
        """Self-supervised pretraining"""
        print("Self-supervised pretraining...")
        
        rot_images, rot_labels = self.create_rotation_dataset(images)
        dataset = AdvancedDataset(rot_images, rot_labels, augment=True)
        loader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=0)
        
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(self.model.parameters(), lr=0.001, weight_decay=0.01)
        
        # Ensure model is on correct device
        self.model = self.model.to(device)
        self.model.train()
        
        for epoch in range(epochs):
            correct = 0
            total = 0
            
            for images_batch, labels_batch in loader:
                # Ensure tensors are on correct device
                images_batch = images_batch.to(device)
                labels_batch = labels_batch.to(device)
                
                optimizer.zero_grad()
                
                if use_amp:
                    with autocast():
                        outputs = self.model(images_batch)
                        loss = criterion(outputs, labels_batch)
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    outputs = self.model(images_batch)
                    loss = criterion(outputs, labels_batch)
                    loss.backward()
                    optimizer.step()
                
                _, predicted = torch.max(outputs, 1)
                total += labels_batch.size(0)
                correct += (predicted == labels_batch).sum().item()
                
                # Memory cleanup
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
            
            if epoch % 10 == 0:
                acc = 100 * correct / total
                print(f"  Pretraining epoch {epoch}: {acc:.1f}%")
        
        print("Pretraining complete")
        return self.model

def create_diverse_models():
    """Create diverse model architectures"""
    models = {}
    
    # Vision Transformers
    try:
        models['vit_base'] = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=5)
        print("Added Vision Transformer")
    except:
        pass
    
    # ConvNeXt
    try:
        models['convnext_tiny'] = timm.create_model('convnext_tiny', pretrained=True, num_classes=5)
        print("Added ConvNeXt")
    except:
        pass
    
    # EfficientNet variants
    try:
        models['efficientnet_v2_s'] = timm.create_model('tf_efficientnetv2_s', pretrained=True, num_classes=5)
        print("Added EfficientNetV2")
    except:
        pass
    
    # RegNet
    try:
        models['regnet_y_800mf'] = timm.create_model('regnetx_800mf', pretrained=True, num_classes=5)
        print("Added RegNet")
    except:
        pass
    
    # Fallback to proven architectures
    models['densenet169'] = timm.create_model('densenet169', pretrained=True, num_classes=5)
    models['resnext50'] = timm.create_model('resnext50_32x4d', pretrained=True, num_classes=5)
    models['densenet201'] = timm.create_model('densenet201', pretrained=True, num_classes=5)
    
    return models

def train_with_all_techniques(model, train_loader, test_loader, model_name, epochs=150):
    """Training with every technique combined"""
    criterion = nn.CrossEntropyLoss(label_smoothing=0.15)
    
    # Different optimizers for different models
    if 'vit' in model_name.lower():
        optimizer = optim.AdamW(model.parameters(), lr=0.0003, weight_decay=0.3)
    elif 'convnext' in model_name.lower():
        optimizer = optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.05)
    else:
        optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=0.05)
    
    # Advanced scheduler with warm restarts
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=30, T_mult=2, eta_min=1e-7
    )
    
    best_acc = 0.0
    patience = 40
    patience_counter = 0
    best_state = None
    
    print(f"Training {model_name} with ALL techniques")
    
    for epoch in range(epochs):
        model.train()
        train_correct = 0
        train_total = 0
        
        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            
            # Random choice of augmentation technique
            aug_choice = random.choice(['mixup', 'cutmix', 'normal'])
            
            if aug_choice == 'mixup' and random.random() < 0.4:
                # Mixup
                lam = np.random.beta(0.4, 0.4)
                batch_size = images.size(0)
                index = torch.randperm(batch_size).to(device)
                mixed_images = lam * images + (1 - lam) * images[index, :]
                y_a, y_b = labels, labels[index]
                
                if use_amp:
                    with autocast():
                        outputs = model(mixed_images)
                        loss = lam * criterion(outputs, y_a) + (1 - lam) * criterion(outputs, y_b)
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    outputs = model(mixed_images)
                    loss = lam * criterion(outputs, y_a) + (1 - lam) * criterion(outputs, y_b)
                    loss.backward()
                    optimizer.step()
                
                _, predicted = torch.max(outputs, 1)
                train_total += labels.size(0)
                train_correct += (predicted == y_a).sum().item()
                
            elif aug_choice == 'cutmix' and random.random() < 0.4:
                # CutMix
                mixed_images, y_a, y_b, lam = cutmix_data(images, labels, alpha=1.0)
                
                if use_amp:
                    with autocast():
                        outputs = model(mixed_images)
                        loss = lam * criterion(outputs, y_a) + (1 - lam) * criterion(outputs, y_b)
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    outputs = model(mixed_images)
                    loss = lam * criterion(outputs, y_a) + (1 - lam) * criterion(outputs, y_b)
                    loss.backward()
                    optimizer.step()
                
                _, predicted = torch.max(outputs, 1)
                train_total += labels.size(0)
                train_correct += (predicted == y_a).sum().item()
                
            else:
                # Normal training
                if use_amp:
                    with autocast():
                        outputs = model(images)
                        loss = criterion(outputs, labels)
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                
                _, predicted = torch.max(outputs, 1)
                train_total += labels.size(0)
                train_correct += (predicted == labels).sum().item()
            
            if batch_idx % 10 == 0 and torch.cuda.is_available():
                torch.cuda.empty_cache()
        
        train_acc = 100 * train_correct / train_total
        scheduler.step()
        
        # Advanced TTA evaluation
        test_acc = evaluate_with_advanced_tta(model, test_loader)
        
        if test_acc > best_acc:
            best_acc = test_acc
            patience_counter = 0
            best_state = model.state_dict().copy()
        else:
            patience_counter += 1
        
        if epoch % 25 == 0 or patience_counter >= patience:
            print(f"  Epoch {epoch:3d}: Train {train_acc:.1f}%, Test+TTA {test_acc:.1f}%")
        
        if patience_counter >= patience:
            print(f"  Early stopping at epoch {epoch}")
            break
        
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    if best_state is not None:
        model.load_state_dict(best_state)
    
    return model, best_acc

def evaluate_with_advanced_tta(model, test_loader, num_tta=6):
    """Advanced TTA with multiple techniques"""
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            
            tta_outputs = []
            
            # Original
            if use_amp:
                with autocast():
                    outputs = model(images)
            else:
                outputs = model(images)
            tta_outputs.append(F.softmax(outputs, dim=1))
            
            # Horizontal flip
            flipped = torch.flip(images, [3])
            if use_amp:
                with autocast():
                    outputs = model(flipped)
            else:
                outputs = model(flipped)
            tta_outputs.append(F.softmax(outputs, dim=1))
            
            # Multiple scales
            for scale in [0.9, 1.1]:
                scaled = F.interpolate(images, scale_factor=scale, mode='bilinear', align_corners=False)
                scaled = F.interpolate(scaled, size=(224, 224), mode='bilinear', align_corners=False)
                if use_amp:
                    with autocast():
                        outputs = model(scaled)
                else:
                    outputs = model(scaled)
                tta_outputs.append(F.softmax(outputs, dim=1))
            
            # Crop variants
            for crop_factor in [0.85, 0.95]:
                size = int(224 * crop_factor)
                start = (224 - size) // 2
                cropped = images[:, :, start:start+size, start:start+size]
                cropped = F.interpolate(cropped, size=(224, 224), mode='bilinear', align_corners=False)
                if use_amp:
                    with autocast():
                        outputs = model(cropped)
                else:
                    outputs = model(cropped)
                tta_outputs.append(F.softmax(outputs, dim=1))
            
            # Average all predictions
            avg_output = torch.stack(tta_outputs).mean(0)
            _, predicted = torch.max(avg_output, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    return 100 * correct / total

def extract_features_for_ml(model, images):
    """Extract CNN features for traditional ML"""
    model.eval()
    features = []
    
    dataset = AdvancedDataset(images, np.zeros(len(images)), augment=False)
    loader = DataLoader(dataset, batch_size=16, shuffle=False, num_workers=0)
    
    with torch.no_grad():
        for imgs, _ in loader:
            imgs = imgs.to(device)
            
            # Remove final classification layer
            if hasattr(model, 'classifier'):
                feat = model.features(imgs)
                feat = F.adaptive_avg_pool2d(feat, (1, 1))
                feat = feat.view(feat.size(0), -1)
            elif hasattr(model, 'fc'):
                feat = model.forward_head(imgs, pre_logits=True)
            else:
                # For ViT and other models
                feat = model.forward_features(imgs)
                if len(feat.shape) > 2:
                    feat = feat.mean(dim=1)
            
            features.append(feat.cpu().numpy())
    
    return np.vstack(features)

def progressive_training(model, X_train, y_train, X_test, y_test, model_name):
    """Progressive training: easy distinctions first"""
    print(f"Progressive training for {model_name}")
    
    # Phase 1: Binary young vs old (classes 0,1,2 vs 3,4)
    y_binary_train = np.array([0 if y <= 2 else 1 for y in y_train])
    y_binary_test = np.array([0 if y <= 2 else 1 for y in y_test])
    
    # Modify model for binary classification
    if hasattr(model, 'classifier'):
        model.classifier = nn.Linear(model.classifier.in_features, 2)
    elif hasattr(model, 'fc'):
        model.fc = nn.Linear(model.fc.in_features, 2)
    
    model = model.to(device)
    
    # Train binary classifier
    binary_dataset_train = AdvancedDataset(X_train, y_binary_train, augment=True)
    binary_dataset_test = AdvancedDataset(X_test, y_binary_test, augment=False)
    binary_loader_train = DataLoader(binary_dataset_train, batch_size=16, shuffle=True, num_workers=0)
    binary_loader_test = DataLoader(binary_dataset_test, batch_size=16, shuffle=False, num_workers=0)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
    
    model.train()
    for epoch in range(30):
        for images, labels in binary_loader_train:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            
            if use_amp:
                with autocast():
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
    
    # Phase 2: Modify for 5-class and fine-tune
    if hasattr(model, 'classifier'):
        model.classifier = nn.Linear(model.classifier.in_features, 5)
    elif hasattr(model, 'fc'):
        model.fc = nn.Linear(model.fc.in_features, 5)
    
    model = model.to(device)
    
    # Fine-tune on all classes
    full_dataset_train = AdvancedDataset(X_train, y_train, augment=True)
    full_dataset_test = AdvancedDataset(X_test, y_test, augment=False)
    full_loader_train = DataLoader(full_dataset_train, batch_size=16, shuffle=True, num_workers=0)
    full_loader_test = DataLoader(full_dataset_test, batch_size=16, shuffle=False, num_workers=0)
    
    # Lower learning rate for fine-tuning
    optimizer = optim.AdamW(model.parameters(), lr=0.0002, weight_decay=0.01)
    
    best_acc = 0.0
    for epoch in range(50):
        model.train()
        for images, labels in full_loader_train:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            
            if use_amp:
                with autocast():
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
        
        # Evaluate
        test_acc = evaluate_with_advanced_tta(model, full_loader_test)
        if test_acc > best_acc:
            best_acc = test_acc
        
        if epoch % 10 == 0:
            print(f"  Progressive epoch {epoch}: {test_acc:.1f}%")
    
    return model, best_acc

# Main comprehensive pipeline
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f"deer_age_comprehensive_{timestamp}"
os.makedirs(output_dir, exist_ok=True)
print(f"Comprehensive results saved to: {output_dir}")

print("Loading images...")
X, y = load_images(color_path, grayscale_path)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train: {len(X_train)}, Test: {len(X_test)}")

# PHASE 1: Advanced Architecture Survey
print("\n" + "="*80)
print("PHASE 1: ADVANCED ARCHITECTURE SURVEY")
print("="*80)

diverse_models = create_diverse_models()
phase1_results = []

for model_name, model_template in diverse_models.items():
    print(f"\n--- Testing {model_name} ---")
    
    try:
        # Self-supervised pretraining
        pretrainer = SelfSupervisedPretrainer(model_template.to(device))
        pretrained_model = pretrainer.pretrain(X_train, epochs=30)
        
        # Reset for main task
        if hasattr(pretrained_model, 'classifier'):
            pretrained_model.classifier = nn.Linear(pretrained_model.classifier.in_features, 5)
        elif hasattr(pretrained_model, 'fc'):
            pretrained_model.fc = nn.Linear(pretrained_model.fc.in_features, 5)
        
        pretrained_model = pretrained_model.to(device)
        
        # Create datasets
        train_dataset = AdvancedDataset(X_train, y_train, augment=True)
        test_dataset = AdvancedDataset(X_test, y_test, augment=False)
        train_loader = DataLoader(train_dataset, batch_size=12, shuffle=True, num_workers=0)
        test_loader = DataLoader(test_dataset, batch_size=12, shuffle=False, num_workers=0)
        
        # Train with all techniques
        trained_model, accuracy = train_with_all_techniques(
            pretrained_model, train_loader, test_loader, model_name, epochs=120
        )
        
        # Save model
        acc_str = f"{accuracy:.1f}".replace('.', 'p')
        model_filename = f"{model_name}_pretrained_{acc_str}pct.pth"
        model_path = os.path.join(output_dir, model_filename)
        
        torch.save({
            'model_state_dict': trained_model.state_dict(),
            'model_name': model_name,
            'accuracy': accuracy,
            'num_classes': 5,
            'pretrained': True
        }, model_path)
        
        phase1_results.append({
            'model': model_name,
            'accuracy': accuracy,
            'filename': model_filename,
            'trained_model': trained_model
        })
        
        print(f"{model_name} with pretraining: {accuracy:.1f}%")
        
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()
        
    except Exception as e:
        print(f"Error with {model_name}: {e}")
        continue

# PHASE 2: Progressive Training
print("\n" + "="*80)
print("PHASE 2: PROGRESSIVE TRAINING")
print("="*80)

phase2_results = []
for model_name in ['densenet169', 'resnext50_32x4d']:
    print(f"\n--- Progressive training {model_name} ---")
    
    try:
        model = timm.create_model(model_name, pretrained=True, num_classes=5)
        trained_model, accuracy = progressive_training(
            model, X_train, y_train, X_test, y_test, model_name
        )
        
        acc_str = f"{accuracy:.1f}".replace('.', 'p')
        model_filename = f"{model_name}_progressive_{acc_str}pct.pth"
        model_path = os.path.join(output_dir, model_filename)
        
        torch.save({
            'model_state_dict': trained_model.state_dict(),
            'model_name': f"{model_name}_progressive",
            'accuracy': accuracy,
            'num_classes': 5
        }, model_path)
        
        phase2_results.append({
            'model': f"{model_name}_progressive",
            'accuracy': accuracy,
            'filename': model_filename,
            'trained_model': trained_model
        })
        
        print(f"{model_name} progressive: {accuracy:.1f}%")
        
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()
        
    except Exception as e:
        print(f"Error with progressive {model_name}: {e}")
        continue

# PHASE 3: CNN Feature Extraction + Traditional ML
print("\n" + "="*80)
print("PHASE 3: HYBRID CNN + TRADITIONAL ML")
print("="*80)

phase3_results = []
if phase1_results:
    # Use best CNN model for feature extraction
    best_cnn = max(phase1_results, key=lambda x: x['accuracy'])
    print(f"Using {best_cnn['model']} for feature extraction...")
    
    # Extract features
    train_features = extract_features_for_ml(best_cnn['trained_model'], X_train)
    test_features = extract_features_for_ml(best_cnn['trained_model'], X_test)
    
    # Traditional ML models
    ml_models = {
        'RandomForest': RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42),
        'XGBoost': XGBClassifier(n_estimators=200, max_depth=6, random_state=42),
        'SVM': SVC(kernel='rbf', C=10, gamma='scale', random_state=42)
    }
    
    for ml_name, ml_model in ml_models.items():
        try:
            print(f"Training {ml_name} on CNN features...")
            ml_model.fit(train_features, y_train)
            ml_predictions = ml_model.predict(test_features)
            ml_accuracy = accuracy_score(y_test, ml_predictions) * 100
            
            phase3_results.append({
                'model': f"CNN+{ml_name}",
                'accuracy': ml_accuracy,
                'type': 'hybrid'
            })
            
            print(f"CNN + {ml_name}: {ml_accuracy:.1f}%")
            
        except Exception as e:
            print(f"Error with {ml_name}: {e}")
            continue

# PHASE 4: Ordinal Regression
print("\n" + "="*80)
print("PHASE 4: ORDINAL REGRESSION")
print("="*80)

phase4_results = []
try:
    print("Training ordinal regression model...")
    
    # Create ordinal targets
    y_train_ordinal = np.array([age_to_ordinal(y_train[i]*0.5 + 1.5) for i in range(len(y_train))])
    y_test_ordinal = np.array([age_to_ordinal(y_test[i]*0.5 + 1.5) for i in range(len(y_test))])
    
    # Remove None values
    valid_train = [i for i, val in enumerate(y_train_ordinal) if val is not None]
    valid_test = [i for i, val in enumerate(y_test_ordinal) if val is not None]
    
    if valid_train and valid_test:
        X_train_ord = X_train[valid_train]
        y_train_ord = np.array([y_train_ordinal[i] for i in valid_train])
        X_test_ord = X_test[valid_test]
        y_test_ord = np.array([y_test_ordinal[i] for i in valid_test])
        
        # Create ordinal model
        ordinal_model = timm.create_model('densenet169', pretrained=True, num_classes=4)
        ordinal_model = ordinal_model.to(device)
        
        # Replace classifier for ordinal regression
        if hasattr(ordinal_model, 'classifier'):
            ordinal_model.classifier = nn.Linear(ordinal_model.classifier.in_features, 4)
        
        # Create datasets
        train_ord_dataset = AdvancedDataset(X_train_ord, np.zeros(len(X_train_ord)), augment=True, ordinal=True)
        test_ord_dataset = AdvancedDataset(X_test_ord, np.zeros(len(X_test_ord)), augment=False, ordinal=True)
        train_ord_loader = DataLoader(train_ord_dataset, batch_size=16, shuffle=True, num_workers=0)
        test_ord_loader = DataLoader(test_ord_dataset, batch_size=16, shuffle=False, num_workers=0)
        
        # Train ordinal model
        ordinal_criterion = OrdinalLoss()
        ordinal_optimizer = optim.AdamW(ordinal_model.parameters(), lr=0.001, weight_decay=0.01)
        
        best_ordinal_acc = 0.0
        for epoch in range(50):
            ordinal_model.train()
            for images, targets in train_ord_loader:
                images = images.to(device)
                # Reconstruct ordinal targets from y_train_ord
                batch_targets = torch.FloatTensor([y_train_ord[i % len(y_train_ord)] for i in range(len(images))]).to(device)
                
                ordinal_optimizer.zero_grad()
                
                if use_amp:
                    with autocast():
                        outputs = ordinal_model(images)
                        loss = ordinal_criterion(outputs, batch_targets)
                    scaler.scale(loss).backward()
                    scaler.step(ordinal_optimizer)
                    scaler.update()
                else:
                    outputs = ordinal_model(images)
                    loss = ordinal_criterion(outputs, batch_targets)
                    loss.backward()
                    ordinal_optimizer.step()
            
            # Evaluate ordinal model (convert back to class predictions)
            ordinal_model.eval()
            correct = 0
            total = 0
            
            with torch.no_grad():
                for images, _ in test_ord_loader:
                    images = images.to(device)
                    outputs = torch.sigmoid(ordinal_model(images))
                    # Convert ordinal predictions back to classes
                    predicted_classes = (outputs > 0.5).sum(dim=1)
                    
                    batch_true_classes = torch.LongTensor([y_test[valid_test[i % len(valid_test)]] for i in range(len(images))]).to(device)
                    
                    total += len(images)
                    correct += (predicted_classes == batch_true_classes).sum().item()
            
            ordinal_acc = 100 * correct / total
            if ordinal_acc > best_ordinal_acc:
                best_ordinal_acc = ordinal_acc
            
            if epoch % 10 == 0:
                print(f"  Ordinal epoch {epoch}: {ordinal_acc:.1f}%")
        
        phase4_results.append({
            'model': 'Ordinal_Regression',
            'accuracy': best_ordinal_acc,
            'type': 'ordinal'
        })
        
        print(f"Ordinal regression: {best_ordinal_acc:.1f}%")

except Exception as e:
    print(f"Error with ordinal regression: {e}")

# FINAL RESULTS COMPILATION
print("\n" + "="*80)
print("COMPREHENSIVE RESULTS SUMMARY")
print("="*80)

all_results = []
all_results.extend(phase1_results)
all_results.extend(phase2_results)
all_results.extend(phase3_results)
all_results.extend(phase4_results)

all_results.sort(key=lambda x: x['accuracy'], reverse=True)

print("PHASE 1 - ADVANCED ARCHITECTURES + PRETRAINING:")
for result in phase1_results:
    print(f"  {result['model']:25s} - {result['accuracy']:.1f}%")

print("\nPHASE 2 - PROGRESSIVE TRAINING:")
for result in phase2_results:
    print(f"  {result['model']:25s} - {result['accuracy']:.1f}%")

print("\nPHASE 3 - HYBRID CNN + TRADITIONAL ML:")
for result in phase3_results:
    print(f"  {result['model']:25s} - {result['accuracy']:.1f}%")

print("\nPHASE 4 - ORDINAL REGRESSION:")
for result in phase4_results:
    print(f"  {result['model']:25s} - {result['accuracy']:.1f}%")

print(f"\n{'='*80}")
print("OVERALL BEST RESULTS:")
for i, result in enumerate(all_results[:10], 1):
    status = "TARGET ACHIEVED" if result['accuracy'] >= 75.0 else f"{75.0-result['accuracy']:.1f}% to go"
    print(f"{i:2d}. {result['model']:25s} - {result['accuracy']:.1f}% - {status}")

if all_results:
    best_overall = all_results[0]
    print(f"\nBEST OVERALL: {best_overall['model']} - {best_overall['accuracy']:.1f}%")
    
    if best_overall['accuracy'] >= 75.0:
        print("SUCCESS: 75% TARGET ACHIEVED!")
    else:
        gap = 75.0 - best_overall['accuracy']
        print(f"Best effort with 466 images: {gap:.1f}% short of 75% target")
        print("Consider multi-fold ensemble if this is insufficient")

print(f"\nAll models and results saved in: {output_dir}")
print("="*80)

Using device: cuda
GPU: NVIDIA GeForce RTX 2060
Mixed Precision: Enabled
ERROR! Session/line number was not unique in database. History logging moved to new session 287
Comprehensive results saved to: deer_age_comprehensive_20250801_075228
Loading images...
Total images: 466
Class distribution: Counter({np.int64(4): 118, np.int64(2): 111, np.int64(3): 89, np.int64(1): 82, np.int64(0): 66})
Train: 372, Test: 94

PHASE 1: ADVANCED ARCHITECTURE SURVEY
Added Vision Transformer
Added ConvNeXt
Added EfficientNetV2

--- Testing vit_base ---
Self-supervised pretraining...
  Pretraining epoch 0: 23.3%
  Pretraining epoch 10: 23.1%
  Pretraining epoch 20: 24.1%
Pretraining complete
Training vit_base with ALL techniques
  Epoch   0: Train 23.7%, Test+TTA 19.1%
  Epoch  25: Train 25.0%, Test+TTA 27.7%
  Epoch  50: Train 23.7%, Test+TTA 25.5%
  Epoch  61: Train 24.7%, Test+TTA 24.5%
  Early stopping at epoch 61
vit_base with pretraining: 27.7%

--- Testing convnext_tiny ---
Self-supervised pretrain

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import timm
import numpy as np
import cv2
import os
import gc
import random
import json
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold, train_test_split
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

try:
    from torch.cuda.amp import autocast, GradScaler
    MIXED_PRECISION_AVAILABLE = True
except ImportError:
    MIXED_PRECISION_AVAILABLE = False
    class autocast:
        def __enter__(self):
            return self
        def __exit__(self, *args):
            pass

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    torch.backends.cudnn.benchmark = True
    if MIXED_PRECISION_AVAILABLE:
        scaler = GradScaler()
        use_amp = True
        print("Mixed Precision: Enabled")
    else:
        use_amp = False
else:
    use_amp = False

color_path = r"G:\Dropbox\AI Projects\buck\images\squared\color"
grayscale_path = r"G:\Dropbox\AI Projects\buck\images\squared\grayscale"

def parse_filename(filename):
    parts = filename.split('_')
    if len(parts) >= 4:
        age_str = parts[3]
        try:
            age = float(age_str.replace('p', '.'))
            if age > 5.5:
                age = 5.5
            return age
        except ValueError:
            return None
    return None

def age_to_class(age):
    age_mapping = {1.5: 0, 2.5: 1, 3.5: 2, 4.5: 3, 5.5: 4}
    return age_mapping.get(age, None)

def load_images(color_path, grayscale_path, img_size=(224, 224)):
    images = []
    ages = []
    
    if os.path.exists(color_path):
        for filename in os.listdir(color_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                age = parse_filename(filename)
                if age is not None:
                    class_idx = age_to_class(age)
                    if class_idx is not None:
                        img_path = os.path.join(color_path, filename)
                        img = cv2.imread(img_path)
                        if img is not None:
                            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                            img_resized = cv2.resize(img_gray, img_size)
                            img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
                            images.append(img_rgb)
                            ages.append(class_idx)
    
    if os.path.exists(grayscale_path):
        for filename in os.listdir(grayscale_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                age = parse_filename(filename)
                if age is not None:
                    class_idx = age_to_class(age)
                    if class_idx is not None:
                        img_path = os.path.join(grayscale_path, filename)
                        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                        if img is not None:
                            img_resized = cv2.resize(img, img_size)
                            img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
                            images.append(img_rgb)
                            ages.append(class_idx)
    
    images = np.array(images)
    ages = np.array(ages)
    
    print(f"Total images: {len(images)}")
    print(f"Class distribution: {Counter(ages)}")
    
    return images, ages

def conservative_augment(image):
    """Conservative augmentation to reduce overfitting"""
    if random.random() < 0.5:
        image = cv2.flip(image, 1)
    
    if random.random() < 0.4:
        angle = random.uniform(-10, 10)
        h, w = image.shape[:2]
        M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1.0)
        image = cv2.warpAffine(image, M, (w, h))
    
    if random.random() < 0.6:
        alpha = random.uniform(0.9, 1.1)
        beta = random.randint(-15, 15)
        image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    
    if random.random() < 0.2:
        noise = np.random.normal(0, 4, image.shape).astype(np.int16)
        image_int16 = image.astype(np.int16)
        noisy_image = np.clip(image_int16 + noise, 0, 255)
        image = noisy_image.astype(np.uint8)
    
    return image

class ConservativeDataset(Dataset):
    def __init__(self, X, y, augment=False):
        self.X = X
        self.y = torch.LongTensor(y)
        self.augment = augment
        self.mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        self.std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        image = self.X[idx].copy()
        label = self.y[idx].clone()
        
        if self.augment:
            image = conservative_augment(image)
        
        image = torch.FloatTensor(image)
        if image.max() > 1.0:
            image = image / 255.0
        
        if len(image.shape) == 3 and image.shape[-1] == 3:
            image = image.permute(2, 0, 1)
        
        image = (image - self.mean) / self.std
        
        return image, label

def create_conservative_model(model_name, num_classes=5):
    """Create model with heavy regularization to combat overfitting"""
    model = timm.create_model(model_name, pretrained=True, num_classes=num_classes, drop_rate=0.6)
    
    # Aggressive freezing to reduce overfitting
    if 'densenet' in model_name:
        for name, param in model.named_parameters():
            if not ('denseblock4' in name or 'classifier' in name):
                param.requires_grad = False
    elif 'resnext' in model_name or 'resnet' in model_name:
        for name, param in model.named_parameters():
            if not ('layer4' in name or 'fc' in name):
                param.requires_grad = False
    elif 'efficientnet' in model_name:
        for name, param in model.named_parameters():
            if not ('blocks.6' in name or 'blocks.7' in name or 'classifier' in name):
                param.requires_grad = False
    
    return model.to(device)

def train_conservative_model(model, train_loader, val_loader, model_name, fold_num):
    """Conservative training to minimize overfitting"""
    criterion = nn.CrossEntropyLoss(label_smoothing=0.2)
    
    # Low learning rate and high weight decay
    optimizer = optim.AdamW(model.parameters(), lr=0.0003, weight_decay=0.1)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=60, eta_min=1e-7)
    
    best_val_acc = 0.0
    patience = 20
    patience_counter = 0
    best_state = None
    
    print(f"    Training {model_name} - Fold {fold_num}")
    
    for epoch in range(60):  # Shorter training to reduce overfitting
        # Training
        model.train()
        train_correct = 0
        train_total = 0
        
        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            
            # Light mixup occasionally
            if random.random() < 0.2:
                lam = np.random.beta(0.2, 0.2)
                batch_size = images.size(0)
                index = torch.randperm(batch_size).to(device)
                mixed_images = lam * images + (1 - lam) * images[index, :]
                y_a, y_b = labels, labels[index]
                
                if use_amp:
                    with autocast():
                        outputs = model(mixed_images)
                        loss = lam * criterion(outputs, y_a) + (1 - lam) * criterion(outputs, y_b)
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    outputs = model(mixed_images)
                    loss = lam * criterion(outputs, y_a) + (1 - lam) * criterion(outputs, y_b)
                    loss.backward()
                    optimizer.step()
                
                _, predicted = torch.max(outputs, 1)
                train_total += labels.size(0)
                train_correct += (predicted == y_a).sum().item()
            else:
                if use_amp:
                    with autocast():
                        outputs = model(images)
                        loss = criterion(outputs, labels)
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                
                _, predicted = torch.max(outputs, 1)
                train_total += labels.size(0)
                train_correct += (predicted == labels).sum().item()
            
            if batch_idx % 10 == 0 and torch.cuda.is_available():
                torch.cuda.empty_cache()
        
        train_acc = 100 * train_correct / train_total
        scheduler.step()
        
        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                
                if use_amp:
                    with autocast():
                        outputs = model(images)
                else:
                    outputs = model(images)
                
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        val_acc = 100 * val_correct / val_total
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            best_state = model.state_dict().copy()
        else:
            patience_counter += 1
        
        if epoch % 15 == 0 or patience_counter >= patience:
            print(f"      Epoch {epoch:2d}: Train {train_acc:.1f}%, Val {val_acc:.1f}%")
        
        if patience_counter >= patience:
            print(f"      Early stopping at epoch {epoch}")
            break
        
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    if best_state is not None:
        model.load_state_dict(best_state)
    
    return model, best_val_acc

def progressive_train_fold(X_fold_train, y_fold_train, X_fold_val, y_fold_val, model_name, fold_num):
    """Progressive training for a single fold"""
    print(f"    Progressive training {model_name} - Fold {fold_num}")
    
    # Phase 1: Binary classification
    y_binary_train = np.array([0 if y <= 2 else 1 for y in y_fold_train])
    y_binary_val = np.array([0 if y <= 2 else 1 for y in y_fold_val])
    
    model = timm.create_model(model_name, pretrained=True, num_classes=2, drop_rate=0.5)
    if hasattr(model, 'classifier'):
        model.classifier = nn.Linear(model.classifier.in_features, 2)
    elif hasattr(model, 'fc'):
        model.fc = nn.Linear(model.fc.in_features, 2)
    model = model.to(device)
    
    # Binary training
    binary_train_dataset = ConservativeDataset(X_fold_train, y_binary_train, augment=True)
    binary_val_dataset = ConservativeDataset(X_fold_val, y_binary_val, augment=False)
    binary_train_loader = DataLoader(binary_train_dataset, batch_size=16, shuffle=True, num_workers=0)
    binary_val_loader = DataLoader(binary_val_dataset, batch_size=16, shuffle=False, num_workers=0)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.05)
    
    for epoch in range(25):
        model.train()
        for images, labels in binary_train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            
            if use_amp:
                with autocast():
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
    
    # Phase 2: 5-class fine-tuning
    if hasattr(model, 'classifier'):
        model.classifier = nn.Linear(model.classifier.in_features, 5)
    elif hasattr(model, 'fc'):
        model.fc = nn.Linear(model.fc.in_features, 5)
    model = model.to(device)
    
    # 5-class datasets
    full_train_dataset = ConservativeDataset(X_fold_train, y_fold_train, augment=True)
    full_val_dataset = ConservativeDataset(X_fold_val, y_fold_val, augment=False)
    full_train_loader = DataLoader(full_train_dataset, batch_size=16, shuffle=True, num_workers=0)
    full_val_loader = DataLoader(full_val_dataset, batch_size=16, shuffle=False, num_workers=0)
    
    # Fine-tune with lower learning rate
    trained_model, val_acc = train_conservative_model(model, full_train_loader, full_val_loader, model_name, fold_num)
    
    return trained_model, val_acc

def evaluate_with_tta(model, test_loader):
    """Simple TTA evaluation"""
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            
            # Original
            if use_amp:
                with autocast():
                    outputs1 = model(images)
            else:
                outputs1 = model(images)
            
            # Horizontal flip
            flipped = torch.flip(images, [3])
            if use_amp:
                with autocast():
                    outputs2 = model(flipped)
            else:
                outputs2 = model(flipped)
            
            # Average predictions
            avg_outputs = (F.softmax(outputs1, dim=1) + F.softmax(outputs2, dim=1)) / 2
            _, predicted = torch.max(avg_outputs, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    return 100 * correct / total

def ensemble_evaluate(all_models, test_loader):
    """Evaluate ensemble of all models from all folds"""
    for models_list in all_models:
        for model in models_list:
            model.eval()
    
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            
            ensemble_output = torch.zeros(images.size(0), 5).to(device)
            model_count = 0
            
            # Collect predictions from all models
            for models_list in all_models:
                for model in models_list:
                    # TTA for each model
                    if use_amp:
                        with autocast():
                            outputs1 = model(images)
                            outputs2 = model(torch.flip(images, [3]))
                    else:
                        outputs1 = model(images)
                        outputs2 = model(torch.flip(images, [3]))
                    
                    avg_model_output = (F.softmax(outputs1, dim=1) + F.softmax(outputs2, dim=1)) / 2
                    ensemble_output += avg_model_output
                    model_count += 1
            
            # Average across all models
            ensemble_output /= model_count
            _, predicted = torch.max(ensemble_output, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    return 100 * correct / total

# Main Multi-Fold Ensemble Pipeline
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f"deer_age_multifold_{timestamp}"
os.makedirs(output_dir, exist_ok=True)
print(f"Multi-fold ensemble results saved to: {output_dir}")

print("Loading images...")
X, y = load_images(color_path, grayscale_path)

# Reserve test set
X_train_all, X_test_final, y_train_all, y_test_final = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training pool: {len(X_train_all)}, Final test: {len(X_test_final)}")

# Model configurations based on best performers
model_configs = [
    ('resnext50_32x4d', 'ResNeXt50'),
    ('tf_efficientnetv2_s', 'EfficientNetV2'),
    ('densenet169', 'DenseNet169')
]

# 5-fold cross-validation
n_folds = 5
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

print(f"\n{'='*80}")
print(f"MULTI-FOLD ENSEMBLE: {n_folds} FOLDS x {len(model_configs)} MODELS")
print(f"{'='*80}")
print("Training strategy: Conservative to minimize overfitting")
print("Total models to train:", n_folds * len(model_configs) * 2)  # x2 for regular + progressive
print("Estimated time: 4-6 hours")

all_trained_models = []
all_fold_scores = []
fold_results = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_all, y_train_all)):
    fold_num = fold + 1
    print(f"\n{'='*60}")
    print(f"FOLD {fold_num}/{n_folds}")
    print(f"{'='*60}")
    
    X_fold_train = X_train_all[train_idx]
    y_fold_train = y_train_all[train_idx]
    X_fold_val = X_train_all[val_idx]
    y_fold_val = y_train_all[val_idx]
    
    print(f"Fold {fold_num}: Train {len(X_fold_train)}, Val {len(X_fold_val)}")
    print(f"Train distribution: {Counter(y_fold_train)}")
    print(f"Val distribution: {Counter(y_fold_val)}")
    
    fold_models = []
    fold_scores = []
    
    for model_timm_name, display_name in model_configs:
        print(f"\n--- {display_name} ---")
        
        try:
            # Regular training
            print(f"  Regular training...")
            model = create_conservative_model(model_timm_name)
            
            train_dataset = ConservativeDataset(X_fold_train, y_fold_train, augment=True)
            val_dataset = ConservativeDataset(X_fold_val, y_fold_val, augment=False)
            train_loader = DataLoader(train_dataset, batch_size=14, shuffle=True, num_workers=0)
            val_loader = DataLoader(val_dataset, batch_size=14, shuffle=False, num_workers=0)
            
            trained_model, val_acc = train_conservative_model(
                model, train_loader, val_loader, display_name, fold_num
            )
            
            fold_models.append(trained_model)
            fold_scores.append(val_acc)
            
            print(f"    {display_name} regular: {val_acc:.1f}%")
            
            # Progressive training
            print(f"  Progressive training...")
            prog_model, prog_acc = progressive_train_fold(
                X_fold_train, y_fold_train, X_fold_val, y_fold_val, 
                model_timm_name, fold_num
            )
            
            fold_models.append(prog_model)
            fold_scores.append(prog_acc)
            
            print(f"    {display_name} progressive: {prog_acc:.1f}%")
            
            # Memory cleanup
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            gc.collect()
            
        except Exception as e:
            print(f"    Error with {display_name}: {e}")
            continue
    
    all_trained_models.append(fold_models)
    all_fold_scores.append(fold_scores)
    
    fold_avg = np.mean(fold_scores) if fold_scores else 0
    fold_results.append(fold_avg)
    print(f"\nFold {fold_num} average: {fold_avg:.1f}%")

# Cross-validation results
print(f"\n{'='*80}")
print("CROSS-VALIDATION RESULTS")
print(f"{'='*80}")

cv_mean = np.mean(fold_results)
cv_std = np.std(fold_results)

print("Fold averages:")
for i, score in enumerate(fold_results, 1):
    print(f"  Fold {i}: {score:.1f}%")

print(f"\nCV Mean: {cv_mean:.1f}% ± {cv_std:.1f}%")

# Final ensemble evaluation on held-out test set
print(f"\n{'='*80}")
print("FINAL ENSEMBLE EVALUATION")
print(f"{'='*80}")

test_dataset = ConservativeDataset(X_test_final, y_test_final, augment=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=0)

print("Evaluating ensemble on held-out test set...")
final_ensemble_accuracy = ensemble_evaluate(all_trained_models, test_loader)

# Save ensemble
ensemble_save_path = os.path.join(output_dir, f"multifold_ensemble_{final_ensemble_accuracy:.1f}pct.pth")
torch.save({
    'all_models_state_dicts': [
        [model.state_dict() for model in fold_models] 
        for fold_models in all_trained_models
    ],
    'model_configs': model_configs,
    'fold_scores': all_fold_scores,
    'cv_mean': cv_mean,
    'cv_std': cv_std,
    'final_test_accuracy': final_ensemble_accuracy,
    'n_folds': n_folds,
    'total_models': sum(len(fold) for fold in all_trained_models)
}, ensemble_save_path)

# Final results
print(f"\n{'='*80}")
print("FINAL RESULTS")
print(f"{'='*80}")

print(f"Cross-validation: {cv_mean:.1f}% ± {cv_std:.1f}%")
print(f"Final ensemble test accuracy: {final_ensemble_accuracy:.1f}%")
print(f"Total models in ensemble: {sum(len(fold) for fold in all_trained_models)}")

if final_ensemble_accuracy >= 75.0:
    print("\nSUCCESS: 75% TARGET ACHIEVED!")
    print("The multi-fold ensemble approach worked!")
else:
    gap = 75.0 - final_ensemble_accuracy
    print(f"\nFinal result: {gap:.1f}% short of 75% target")
    if final_ensemble_accuracy >= 70.0:
        print("Very close! This is likely the ceiling with 466 images.")
    else:
        print("The dataset size appears to be the fundamental limiting factor.")

print(f"\nEnsemble saved: {ensemble_save_path}")
print("="*80)

# Save detailed results
results_summary = {
    'timestamp': timestamp,
    'total_images': len(X),
    'train_images': len(X_train_all),
    'test_images': len(X_test_final),
    'n_folds': n_folds,
    'model_configs': model_configs,
    'fold_results': fold_results,
    'cv_mean': float(cv_mean),
    'cv_std': float(cv_std),
    'final_test_accuracy': float(final_ensemble_accuracy),
    'target_achieved': final_ensemble_accuracy >= 75.0,
    'total_models': sum(len(fold) for fold in all_trained_models)
}

with open(os.path.join(output_dir, 'results_summary.json'), 'w') as f:
    json.dump(results_summary, f, indent=2)

print(f"Detailed results saved to: {os.path.join(output_dir, 'results_summary.json')}")

Using device: cuda
GPU: NVIDIA GeForce RTX 2060
Mixed Precision: Enabled
Multi-fold ensemble results saved to: deer_age_multifold_20250801_223542
Loading images...
Total images: 466
Class distribution: Counter({np.int64(4): 118, np.int64(2): 111, np.int64(3): 89, np.int64(1): 82, np.int64(0): 66})
Training pool: 372, Final test: 94

MULTI-FOLD ENSEMBLE: 5 FOLDS x 3 MODELS
Training strategy: Conservative to minimize overfitting
Total models to train: 30
Estimated time: 4-6 hours

FOLD 1/5
Fold 1: Train 297, Val 75
Train distribution: Counter({np.int64(4): 75, np.int64(2): 71, np.int64(3): 57, np.int64(1): 52, np.int64(0): 42})
Val distribution: Counter({np.int64(4): 19, np.int64(2): 18, np.int64(3): 14, np.int64(1): 13, np.int64(0): 11})

--- ResNeXt50 ---
  Regular training...
    Training ResNeXt50 - Fold 1
      Epoch  0: Train 22.2%, Val 22.7%
      Epoch 15: Train 85.2%, Val 53.3%
      Epoch 30: Train 90.2%, Val 57.3%
      Epoch 45: Train 91.9%, Val 54.7%
      Epoch 46: Train 81